In [1]:
import torch
import torch.optim as optim
import numpy as np
from transformer import Transformer
from metrics import Evaluator
from datasets import load_dataset
from utils import *
from training import *
from tokenizer import *

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device}")

/Users/tonimo/Documents/Developing/Repos/Deep-Learning/deep-learning-venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using cpu


In [2]:
traindict = load_dataset("opus100", "de-en", split="train")
testdict = load_dataset("opus100", "de-en", split="test")

Found cached dataset opus100 (/Users/tonimo/.cache/huggingface/datasets/opus100/de-en/0.0.0/256f3196b69901fb0c79810ef468e2c4ed84fbd563719920b1ff1fdc750f7704)
Found cached dataset opus100 (/Users/tonimo/.cache/huggingface/datasets/opus100/de-en/0.0.0/256f3196b69901fb0c79810ef468e2c4ed84fbd563719920b1ff1fdc750f7704)


In [3]:
train_inputs, train_labels = get_split(traindict, "en", "de", size=100)
test_inputs, test_labels = get_split(testdict, "en", "de", size=10)
trainset = Dataset(train_inputs, train_labels)
testset = Dataset(test_inputs, test_labels)

In [4]:
trainframe = trainset.dataframe()
trainframe.head()

,inputs,labels
0,It's greed that it's gonna be the death of you...,Deine Habgier wird noch dein Tod sein.
1,Vega.,- Vega.
2,Just say when.,Sagen Sie einfach stopp.
3,- Wait.,- Warte.
4,I don't wanna be here.,Ich will nicht hier sein.


In [5]:
testframe = testset.dataframe()
testframe.head()

,inputs,labels
0,"By clicking on 'Save profile', you the user ag...",Die Nutzungsbedingungen werden durch das Klick...
1,I wanted to show you something first.,Ich wollte dir erst noch etwas zeigen.
2,You have suffered because of Shinkichi.,Du musstest wegen Shinkichi leiden.
3,"moodle:bg-bab: Calendar: Day view: Friday, 25 ...",moodle:bg-bab: Kalender: Tagesansicht: Freitag...
4,"I mean, most people, they see another person w...","Ich meine, die meisten Leuten sehen eine ander..."


In [6]:
print(trainframe.isnull().values.any())
trainframe.describe()

False


,inputs,labels
count,100,100
unique,100,100
top,It's greed that it's gonna be the death of you...,Deine Habgier wird noch dein Tod sein.
freq,1,1


In [7]:
print(testframe.isnull().values.any())
testframe.describe()

False


,inputs,labels
count,10,10
unique,10,10
top,"By clicking on 'Save profile', you the user ag...",Die Nutzungsbedingungen werden durch das Klick...
freq,1,1


In [8]:
trainset.sample()

[('2009-06-10, 05:45 PM #1', '2011-02-05, 06:02 PM #1')]

In [9]:
testset.sample()

[("So I have to withdraw as Bobby's attorney.",
  'Aus diesem Grund muss ich das Mandat für Bobby niederlegen.')]

In [10]:
corpus_en = trainset.corpus(split=0) + testset.corpus(split=0)
corpus_de = trainset.corpus(split=1) + testset.corpus(split=1)
tokenizer_en = Nerdimizer()
tokenizer_de = Nerdimizer()
tokenizer_en.train(corpus_en, size=50000)
tokenizer_de.train(corpus_de, size=50000)

In [11]:
en_vocab = len(tokenizer_de)
de_vocab = len(tokenizer_de)
maxlen_train = trainset.maxlen(tokenizer_en, tokenizer_de)
maxlen_test = testset.maxlen(tokenizer_en, tokenizer_de)
maxlen = min(maxlen_train, maxlen_test, 200)
start, end, pad = tokenizer_en["[S]"], tokenizer_en["[E]"], tokenizer_en["[P]"]
print(f"Number of input tokens: {len(tokenizer_en)}\nNumber of output tokens: {len(tokenizer_de)}")

Number of input tokens: 860
Number of output tokens: 1066


In [12]:
tokenizer_en.padon(maxlen, pad_id=pad, end=True)
tokenizer_en.truncon(maxlen, end=True)
tokenizer_de.padon(maxlen, pad_id=pad, end=True)
tokenizer_de.truncon(maxlen, end=True)
print(f"Maxlen: {maxlen}")

Maxlen: 46


In [13]:
tokenized_trainset = trainset.tokenized(tokenizer_en, tokenizer_de, model=True)
dataloader = tokenized_trainset.dataloader(batch_size=32, drop_last=False)

In [14]:
model = Transformer(en_vocab, de_vocab, maxlen, pad_id=pad, dm=256, nhead=8, layers=3, dff=1024)
optimizer = optim.Adam(model.parameters(), lr=1e-5, betas=(0.9, 0.98), eps=10e-9)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, patience=10)
evaluator = Evaluator(testset, tokenizer_en, tokenizer_de, start, end, maxlen, sample=10, ngrams=4, threshold=20, 
                    mode="geometric", device=device)
checkpoint = Checkpoint(model, optimizer, scheduler, evaluator, epochs=3, 
                    path="checkpoints/english-german", overwrite=False, verbose=True)
clock = Clock()
model.to(device);

In [15]:
train(dataloader, model, optimizer, scheduler, evaluator, checkpoint, clock,
    epochs=1000, warmups=100, verbose=True, device=device)

Training Started
Epoch 1 Complete | Epoch Duration: 00:00:09 | Elapsed Time: 00:00:09 |
Metrics | Epoch Loss: 7.0239 | BLEU Score: 0.0 | 
Other Info | Scheduler Warmup Step: True | Checkpoint Saved: False |
Epoch 2 Complete | Epoch Duration: 00:00:07 | Elapsed Time: 00:00:26 |
Metrics | Epoch Loss: 6.8474 | BLEU Score: 0.0 | 
Other Info | Scheduler Warmup Step: True | Checkpoint Saved: False |
Checkpoint saved
Epoch 3 Complete | Epoch Duration: 00:00:02 | Elapsed Time: 00:00:44 |
Metrics | Epoch Loss: 6.6631 | BLEU Score: 0.0 | 
Other Info | Scheduler Warmup Step: True | Checkpoint Saved: True |
Epoch 4 Complete | Epoch Duration: 00:00:02 | Elapsed Time: 00:01:05 |
Metrics | Epoch Loss: 6.5241 | BLEU Score: 0.0 | 
Other Info | Scheduler Warmup Step: True | Checkpoint Saved: False |
Epoch 5 Complete | Epoch Duration: 00:00:02 | Elapsed Time: 00:01:27 |
Metrics | Epoch Loss: 6.4400 | BLEU Score: 0.0 | 
Other Info | Scheduler Warmup Step: True | Checkpoint Saved: False |
Checkpoint saved


KeyboardInterrupt: 

In [16]:
checkpoint = Checkpoint(model, optimizer, scheduler, None, None, None, None, None)
checkpoint.load_checkpoint(path="checkpoints/english-german-3")

In [ ]:
retrain(dataloader, checkpoint, 1000)

Training Resumed


TypeError: unsupported operand type(s) for +=: 'NoneType' and 'float'